In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib qt

nx = 9
ny = 6

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

In [ ]:
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

In [ ]:

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    # filter out 200 pixels to the left and right, 0:200 and histogram.shape[0]-200:end
    histogram[0:200] = 0
    histogram[histogram.shape[0]-200:] = 0
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
#     plt.plot(histogram)
#     plt.show()


    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    print("nonzero: ", nonzero)
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    print("nonzerox: ", nonzerox )
    print("nonzeroy: ", nonzeroy )

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    print(binary_warped.shape)

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low =   leftx_current - margin  # Update this
        win_xleft_high =  leftx_current + margin  # Update this
        
        win_xright_low =  rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin# Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds =  ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty



def fit_polynomial(binary_warped, distance_avg):
    print("In fit_polynomial()")
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    
    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    adding = False
    if len(rightx) < 1500:
        adding = True
        
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    if adding:
        print("adding: ", distance_avg)
        right_fitx = left_fitx + distance_avg

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img, left_fit, right_fit, left_fitx, right_fitx, ploty, lefty

def draw_lines(img, undist, left_fitx, right_fitx, ploty, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.imshow("color_warp", color_warp)
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    return result
   
def search_around_poly(binary_warped, left_fit, right_fit, distance_avg):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    print("max(nonzero[0]: ", max(nonzero[0]))
    print("max(nonzero[1]: ", max(nonzero[1]))
     
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    a = ((nonzerox > 200) & (nonzerox < 1100))
    nonzeroy = nonzeroy[a]
    nonzerox = nonzerox[a]
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    win_x_low_left = left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] - margin
    win_x_high_left = left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] + margin
    
    win_x_low_right = right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] - margin
    win_x_high_right = right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] + margin
     
    
    good_left_inds =  ((nonzerox >= win_x_low_left) & (nonzerox < win_x_high_left)) 
    good_right_inds = ((nonzerox >= win_x_low_right) & (nonzerox < win_x_high_right)) 
    
    left_lane_inds = good_left_inds
    right_lane_inds = good_right_inds
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    print("len(leftx): ", len(leftx))
    print("len(rightx): ", len(rightx))
    
    adding = False
    if len(rightx) < 1500:
        rightx = leftx
        righty = lefty
        adding = True

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    if adding:
        print("adding 537: ", distance_avg)
        right_fitx = left_fitx + distance_avg
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.show()
    ## End visualization steps ##
    
    return result, left_fitx, right_fitx, ploty



def measure_curvature_real(leftx, rightx, ploty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    #ploty, left_fit_cr, right_fit_cr = generate_data(ym_per_pix, xm_per_pix)
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    A_left = left_fit_cr[0]
    A_right = right_fit_cr[0]
    
    B_left = left_fit_cr[1]
    B_right = right_fit_cr[1]
    
    y = y_eval*ym_per_pix
    
    left_curverad = ((1+(2*A_left*y + B_left)**2)**1.5)/ np.absolute(2*A_left)  ## Implement the calculation of the left line here
    right_curverad = ((1+(2*A_right*y+B_right)**2)**1.5)/ np.absolute(2*A_right)  ## Implement the calculation of the right line here
   
#    left_curverad = 0  ## Implement the calculation of the left line here
#    right_curverad = 0  ## Implement the calculation of the right line here
    
    return left_curverad, right_curverad

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
#        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
#        cv2.imshow('img',img)
#        cv2.waitKey(100)


ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# print("mtx: " ,mtx)
# print("dist: " ,dist)

# Read in an image
img = cv2.imread('camera_cal/calibration1.jpg')
print(img.shape)
plt.imshow(img)
plt.show()
plt.savefig("distorted.jpg")
undistorted = cal_undistort(img, objpoints, imgpoints)
plt.imshow(undistorted)
plt.savefig("undistorted.jpg")


# cv2.imshow('img',undistorted)
# cv2.waitKey(1000)
# print("undistorted")
# cv2.destroyAllWindows()


In [ ]:
def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    image_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    channel_s = image_hls[:,:,2]
    # 3) Return a binary image of threshold result
    binary = np.zeros_like(channel_s)
    binary[(channel_s > thresh[0]) & (channel_s <= thresh[1])] = 1
    binary_output = binary # placeholder line
    return binary_output

def rgb_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    # image_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    channel_s = img[:,:,0]
    # 3) Return a binary image of threshold result
    binary = np.zeros_like(channel_s)
    binary[(channel_s > thresh[0]) & (channel_s <= thresh[1])] = 1
    binary_output = binary # placeholder line
    return binary_output

def corners_unwarp(undistorted_img):
    # c) define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    #dst = np.float32([[0,0],[undistorted_img.shape[1]-100,100],[undistorted_img.shape[1]-100,undistorted_img.shape[0]-100],[100,undistorted_img.shape[0]-100]])

    print("img.shape[0]", undistorted_img.shape[0])
    print("img.shape[1]", undistorted_img.shape[1])
    src  =  np.float32([[525, undistorted_img.shape[0]//2 + 90], [undistorted_img.shape[1] - 525, undistorted_img.shape[0]//2 + 90], [undistorted_img.shape[1], undistorted_img.shape[0]], [0, undistorted_img.shape[0]]])
    dst  =  np.float32([[0, 0],                                  [undistorted_img.shape[1], 0],                                      [undistorted_img.shape[1]-280, undistorted_img.shape[0]], [280, undistorted_img.shape[0]]])
    
    # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    print("M: ", M)
    # e) use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(undistorted_img, M, (undistorted_img.shape[1], undistorted_img.shape[0]), flags=cv2.INTER_LINEAR)
    
    return warped, M

In [ ]:
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [ ]:
img = mpimg.imread("./test_images/test3.jpg")

undistorted_img = cal_undistort(img, objpoints, imgpoints)

hls_binary = hls_select(undistorted_img, thresh=(100, 255))
hls_binary = hls_binary*255

plt.imshow(hls_binary, cmap='gray')
#plt.savefig("hls.jpg")
plt.show()

top_down, perspective_M = corners_unwarp(hls_binary)
plt.imshow(top_down, cmap='gray')
#plt.savefig("top_down.jpg")
plt.show()


[out_img,  left_fit, right_fit, left_fitx, right_fitx, ploty, lefty] = fit_polynomial(top_down, 0)
plt.imshow(out_img, cmap='gray')
#plt.savefig("sliding_window.jpg")
plt.show()

result = draw_lines( top_down, img, left_fitx, right_fitx, ploty, np.linalg.inv(perspective_M) )
plt.imshow(result, cmap='gray')
plt.savefig("projected_back.jpg")
plt.show()



# image_gaussian = gaussian_blur(hls_binary, 5)

# # detect edges
# low_threshold = 150
# high_threshold = 200

# image_canny = canny(image_gaussian, low_threshold, high_threshold)
# plt.imshow(image_canny, cmap='gray')

In [ ]:
#####################################################
## Images 
####################################################
#img = cv2.imread('test_images/straight_lines1.jpg')
img = cv2.imread('test_images/straight_lines1.jpg')
undistorted_img = cal_undistort(img, objpoints, imgpoints)
cv2.imshow('img_orig',undistorted_img)
cv2.waitKey(1000)

#rgb = img[...,::-1].copy()
hls_binary = hls_select(undistorted_img, thresh=(70, 255))
hls_binary = hls_binary*255
cv2.imshow('img_binary', hls_binary)
cv2.waitKey(1000)

top_down, perspective_M = corners_unwarp(hls_binary)
cv2.imshow('img_unwarped',top_down)
cv2.waitKey(1000)
plt.imshow(top_down, cmap='gray')
plt.savefig("top_down.jpg")

[out_img,  left_fit, right_fit, left_fitx, right_fitx, ploty, lefty] = fit_polynomial(top_down, 0)
cv2.imshow("window", out_img)


result = draw_lines( top_down, img, left_fitx, right_fitx, ploty, np.linalg.inv(perspective_M) )
cv2.imshow("plot lines onto image", result)


print("left_fit, right_fit: ", left_fit, right_fit)
[result, left_fitx, right_fitx, ploty] = search_around_poly(top_down, left_fit, right_fit, 0)
# View your output
plt.imshow(result)

# Calculate the radius of curvature in meters for both lane lines
left_curverad, right_curverad = measure_curvature_real(left_fitx, right_fitx, ploty)
print("left_curverad, right_curverad: ", left_curverad, right_curverad)

# print("left_fitx: ", left_fitx)
# print("ploty: ", ploty)

distance = (right_fitx[-1] - left_fitx[-1])*xm_per_pix
print("distance[m]: ", distance)

In [ ]:
#####################################################
## Video 
####################################################

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

initialized = False
initialized_filter = False
left_fit, right_fit, left_fitx, right_fitx, ploty, distance = None, None, None, None, None, None
prev_left_fitx = None
prev_right_fitx = None
prev_ploty = None
distance_avg = None

def process_image(image):
    global initialized
    global left_fit
    global right_fit
    global left_fitx
    global right_fitx
    global ploty
    global prev_left_fitx
    global prev_right_fitx
    global prev_ploty
    global initialized_filter
    global distance_avg
    
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    img = image
    
    undistorted_img = cal_undistort(img, objpoints, imgpoints) 

    #rgb = img[...,::-1].copy()
    hls_binary = hls_select(undistorted_img, thresh=(220, 255))
    rgb_binary = rgb_select(undistorted_img, thresh=(220, 255))
    #hls_binary = hls_select(img, thresh=(150, 255))
    
    combined_binary = hls_binary |  rgb_binary
    
    #hls_binary = hls_binary*255
    hls_binary = combined_binary*255 
    
    plt.imshow(hls_binary, cmap='gray')
    plt.show()

    top_down, perspective_M = corners_unwarp(hls_binary)
    
    if initialized:
        [result, left_fitx, right_fitx, ploty] = search_around_poly(top_down, left_fit, right_fit, distance_avg)
        left_curverad, right_curverad = measure_curvature_real(left_fitx, right_fitx, ploty)
        print("initialized, left_curverad, right_curverad: ", left_curverad, right_curverad)
        
        factor = 2.5
        if left_curverad * factor < right_curverad:
            print("left_curverad * factor < right_curvard")
            initialized = False 
        
        if right_curverad * factor < left_curverad:
            print("right_curverad * factor < left_curvard")
            initialized = False
        

    if not initialized:
        [out_img,  left_fit, right_fit, left_fitx, right_fitx, ploty, lefty] = fit_polynomial(top_down, distance_avg)
        result = out_img
        initialized = True
    
    if not initialized_filter:
        prev_left_fitx = left_fitx 
        prev_right_fitx = right_fitx
        prev_ploty = ploty
        distance1 = (right_fitx[0] - left_fitx[0])
        distance2 = (right_fitx[-1] - left_fitx[-1])
        distance_avg = (distance1 + distance2)//2
        initialized_filter = True

    plt.imshow(result)
    plt.show()
        
    distance1 = (right_fitx[0] - left_fitx[0])#*xm_per_pix
    distance2 = (right_fitx[-1] - left_fitx[-1])#*xm_per_pix
    distance_avg = (distance1 + distance2 + distance_avg)//3
    print("distance: ", distance1, " ", distance2, "  ", distance_avg)
    print("distance[m]: ", distance1 *xm_per_pix , " ", distance2*xm_per_pix, "  ", distance_avg*xm_per_pix)
    print("center: ")
    
    image_center = image.shape[1]//2
    current_pos = (right_fitx[-1] + left_fitx[-1]) // 2
    dist_from_center = image_center - current_pos
    
    
    # Calculate the radius of curvature in meters for both lane lines
    left_curverad, right_curverad = measure_curvature_real(left_fitx, right_fitx, ploty)
    print("left_curverad, right_curverad: ", left_curverad, right_curverad)

    if np.abs(distance1 - distance2) > 150:
        print("Using previous")
        result = draw_lines( top_down, img, prev_left_fitx, prev_right_fitx, prev_ploty, np.linalg.inv(perspective_M) )
        initialized = False
    else:
        prev_left_fitx = (left_fitx + prev_left_fitx)//2
        prev_right_fitx = (right_fitx + prev_right_fitx)//2
        prev_ploty = (ploty + prev_ploty)//2
        
        # Filter 
        result = draw_lines( top_down, img, (left_fitx + prev_left_fitx)//2, (right_fitx + prev_right_fitx)//2, (ploty+prev_ploty)//2, np.linalg.inv(perspective_M) )
    
    cv2.putText(result, "left_curverad: " + str.format("{0:.2f}", left_curverad), (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255, 255), 3)
    cv2.putText(result, "right_curverad: " + str.format("{0:.2f}", right_curverad), (100, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255, 255), 3)
    cv2.putText(result, "dist_from_center[m]: " + str.format("{0:.2f}", dist_from_center *xm_per_pix), (100, 170), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255, 255), 3)

    return result

In [ ]:
white_output = 'test_videos_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("project_video.mp4").subclip(39,41)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))